* **출처 : [유튜브] 데이터 스테이션 : [Python 활용 데이터 분석 최적화] 05 회귀분석 실습 (사이킷 런 Scikit Learn)**

In [16]:
# 라이브러리 호출 
import pandas as pd
import numpy as np
import plotly.express as px

In [17]:
df1 = pd.read_csv('Data_process.csv')
df1 # 생산 데이터 

,Unnamed: 0,순번,생산라인코드,생산라인명,생산일자,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,실링압력,생산시간
0,0,3,C001,쿠커 #1,2011-01-03,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,201.844,70.37
1,1,1,C002,쿠커 #2,2011-01-03,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,212.043,70.83
2,2,13,C003,쿠커 #3,2011-01-03,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,208.781,73.55
3,3,7,C005,2동 쿠커 #4,2011-01-03,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,208.269,74.81
4,4,2,C007,프리머 #1,2011-01-03,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179319,114.079318,149.928068,23.899,211.625,73.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22904,24059,3,D087,냉동밥 내포장#2,2021-10-28,1BA03638,(1)Homeplus signature 날치알이톡톡터지는날치알밥(알밥),W005,71.288798,102.688798,138.091898,23.602,206.561,72.49
22905,24062,9,D088,냉동밥 내포장#1,2021-10-29,1BA03515,(쿠팡)곰곰갈비맛치킨밥,W005,69.732381,98.432381,133.916831,23.656,213.413,76.63
22906,24063,10,D088,냉동밥 내포장#1,2021-10-29,1BA03612,무명식당뿌리채소영양밥3kg,W005,71.459932,107.559932,143.598182,24.026,215.243,81.00
22907,24067,4,D088,냉동밥 내포장#1,2021-10-29,1BA03606,콩콩푸드밥3kg,W005,70.664452,65.764452,100.773102,23.339,209.513,72.12


#### **생산 시간 예측**

- '쿠킹스팀입력' 값을 넣었을 때, '실링압력'이 어떻게 될지 예측하는 수식 생성
- Y(실링압력) = mX(쿠킹스팅입력) + b
- 최소제곱법 + 경사하강법 두 기법을 활용해 수식 생성 
- 수식 생성 절차 
    1. 파라미터(m, b/ 회귀계수 및 절편)를 임의의 값으로 설정(초기화), 첫 RSS계산(최소제곱법) 
    2. 기존의 파라미터(m, b/ 회귀계수 및 절편) 값을 Update(경사하강법) 후 다시 RSS 계산
    3. RSS 감소했다면, 다시 2번 절차를 반복
       RSS 감소하지 않는다면, 파라미터 확정하고 Update 중단 

**시각화**

In [18]:
px.scatter(df1, x = '쿠킹스팀압력', y = '실링압력')

**함수**

In [19]:
# 3가지 함수 구성
# 1. Parameter를 Update하는 함수 
# 2. 점진적 반복 계산을 수행할 함수 
# 3. RSS 오차항(실제값과 예측값의 차이)를 계산하는 함수 

In [25]:
# 1. Parameter를 Update하는 함수 
def parameter_update(m, b, X, Y, learning_rate) :
    # 데이터를 입력 받은 X, Y, learning_rate와 함께 m , b 값을 초기화해서 update해주는 형식 구성 
    # m값과 b값의 update값을 0값으로 초기화 (np.zeros_like : 초기화)
    m_param = np.zeros_like(m) 
    b_param = np.zeros_like(b) 
    
    # Y = m X + b 값을 계산 (np.dot : 행렬형태)
    Y_pred = np.dot(X, m.T) + b 
    
    # 잔차 계산 (Residual)
    residual = Y - Y_pred
    
    # 행렬 연산을 위한 factor 형태 생성 
    # b_param값을 행렬형태로 연산하여 구하기 위해 1로만 구성된 행렬 생성
    b_frame = np.ones((len(Y), 1))
    
    # 파라미터(m값과 b값 /  회귀계수)값의 변화량 계산
    m_param = -(2/len(Y)) * learning_rate * (np.dot(X.T, residual))
    b_param = -(2/len(Y)) * learning_rate * (np.dot(b_frame.T, residual))
    
    return m_param, b_param

In [26]:
# 2. 점진적 반복 계산을 수행할 함수 
def gradient_step(X, Y, iters) :
    # m, b 값을 0으로 초기화 
    m = np.zeros((1, 1))
    b = np.zeros((1, 1))
    # iters (반복횟수) 만큼 parameter_update 절차를 거쳐, update 수행 
    for i in range(iters) :
        m_param, b_param = parameter_update(m, b, X, Y, 0.001 )
        m = m - m_param
        b = b - b_param
    return m, b

In [27]:
# 3. RSS 오차항(실제값과 예측값의 차이)를 계산하는 함수 
def get_residual(Y, Y_pred) :
    result = np.sum(np.square(Y- Y_pred)) / len(Y)
    return result

In [28]:
# '쿠킹스팀압력' 데이터와 '실링압력' 데이터를 행렬형태로 변환하여 X, Y로 선언
X = df1['쿠킹스팀압력'].values.reshape(-1, 1)
Y = df1['실링압력'].values.reshape(-1, 1)

In [29]:
gradient_step(X, Y, 100)

(array([[8.76559923]]), array([[0.36944495]]))

**Y(실링압력) = 8.7656 X (쿠킹스팀압력) + 0.369**

In [30]:
M, b = gradient_step(X, Y, 100)

y_pred = M[0,0] * X + b 
y_pred

array([[204.73939096],
       [203.95925262],
       [206.5012764 ],
       ...,
       [210.97173201],
       [204.94976534],
       [205.09878052]])